In [ ]:
!pip install fasttext
!pip install huggingface_hub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.8/68.8 kB 1.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached pybind11-2.11.1-py3-none-any.whl (227 kB)
  Created wheel for fasttext: filename=fasttext-0.9.2-cp310-cp310-linux_x86_64.whl size=4199772 sha256=781b972522434e44d468c4129f837e11aa7d676cf1149d51dc6a277b1171426c
  Stored in directory: /root/.cache/pip/wheels/a5/13/75/f811c84a8ab36eedbaef977a6a58a98990e8e0f1967f98f394
Successfully built fasttext
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 5.2 MB/s eta 0:00:00


In [ ]:
import nltk
nltk.download("stopwords")

# import nltk
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
!python -m spacy download el_core_news_sm
!python -m spacy validate

2023-10-30 18:54:49.967225: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-10-30 18:54:49.967299: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-10-30 18:54:49.967333: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-10-30 18:54:49.976373: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-30 18:54:51.238995: W tensorflow/compiler/

In [ ]:
import pandas as pd
import numpy as np

# PRE-PROCESSING
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import spacy

# FastText
from nltk.tokenize import word_tokenize
from gensim.models import FastText

# TRAIN/TEST SET
from sklearn.model_selection import StratifiedKFold, train_test_split

from sklearn import model_selection, naive_bayes, svm
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

nlp = spacy.load("el_core_news_sm")

In [ ]:
# testing  = pd.read_csv('TWEETS_nolinks.csv')
testing  = pd.read_csv('CHATGPT TWEETS_nolinks.csv')

# Replace empty strings with NaN
testing['class'] = testing['class'].replace(' ', np.nan)
final = testing[testing['class'].notnull()]
final = final.reset_index()
final = final[['Tweet', 'class']]
# Use astype to convert to float
final['class'] = final['class'].astype(float)
final.head()

,Tweet,class
0,Η απορία της ημέρας!!\n\nΓιατί σε φωτογραφία π...,0.0
1,Ρε κόψτε την σύνδεση με τον βλακα Καραγιάννη έ...,1.0
2,Ότι ήσουν μαλάκω το ξέραμε. Ότι ήσουν αμπαλη ...,1.0
3,ΠΟΙΟΣ ΓΑΜΙΟΛΗΣ ΚΡΥΒΕΤΑΙ ΠΙΣΩ ΑΠΤΗΝΕ ΑΝΑΠΑΤΕΧΗ ...,1.0
4,Βλεπω αυτόν τον πατέρα στην Σκατιανα και μου έ...,1.0


https://huggingface.co/facebook/fasttext-el-vectors/blob/main/README.md?code=true

In [ ]:
import fasttext
from huggingface_hub import hf_hub_download
model_path = hf_hub_download(repo_id="facebook/fasttext-el-vectors", filename="model.bin")
model = fasttext.load_model(model_path)
# model.words

model.bin:   0%|          | 0.00/7.25G [00:00<?, ?B/s]

In [ ]:
model.get_dimension()


300

In [ ]:
len(model['καλα'])

300

In [ ]:
def get_embedding(word):
    try:
        embedding=model[word]
    except:
        embedding=np.zeros((300,))
    return embedding

In [ ]:
def replaceMultiple(main, replacements, new):
    for elem in replacements:
        if elem in main:
            main = main.replace(elem, new)

    return main


def normalize(x):
    x = x.replace('ά', 'α')
    x = x.replace('έ', 'ε')
    x = x.replace('ή', 'η')
    x = replaceMultiple(x, ['ί', 'ΐ', 'ϊ'], 'ι')
    x = x.replace('ό', 'ο')
    x = replaceMultiple(x, ['ύ', 'ΰ', 'ϋ'], 'υ')
    x = x.replace('ώ', 'ω')
    return x

def remove_stopwords(text):
    stop_words = stopwords.words('greek')

    imp_words = []

    # Storing the important words
    for word in str(text).split():

        if word not in stop_words:

            # Let's Lemmatize the word as well -- to fernw stin arxiki tou morfi
#             before appending to the imp_words list.

            lemmatizer = WordNetLemmatizer()
            lemmatizer.lemmatize(word) #doesnt really work

            imp_words.append(word)

    output = " ".join(imp_words)

    return output


punctuations_list = string.punctuation
def remove_punctuations(text):
    temp = str.maketrans('', '', punctuations_list)
    return text.translate(temp)

def text_processing(text):
    text = normalize(text)  #diwxnw tonous
    text = text.lower() #ola mikra
    text = remove_punctuations(text) #diwxnw punctiations
    text = remove_stopwords(text) #diwxnw stopwords
    text  = " ".join([w.lemma_ for w in nlp(text)]) #lemmatization is not really working well either
    return text
#     return [word for word in text.split() ] #tokenization



In [ ]:
def tokenize(text):
    return [word for word in text.split() ] #tokenization

In [ ]:
final['Tweet'] = final['Tweet'].apply(lambda x: text_processing(x))

In [ ]:
kfold = StratifiedKFold(n_splits=5, random_state=10, shuffle=True)
for i, (train_index, test_index) in enumerate(kfold.split(final['Tweet'], final['class'])):
        X_train, X_test, Y_train, Y_test = final['Tweet'][train_index], final['Tweet'][test_index], final['class'][train_index], final['class'][test_index]

In [ ]:
X_train = [tokenize(tweet) for tweet in X_train]
X_test = [tokenize(tweet) for tweet in X_test]

In [ ]:
X_train_embeddings=[np.mean(np.array(list(map(get_embedding,tok_sent))),axis=0) for tok_sent in X_train]
X_test_embeddings=[np.mean(np.array(list(map(get_embedding,tok_sent))),axis=0) for tok_sent in X_test]

In [ ]:
X_train_embeddings

[array([ 9.75625683e-03, -2.17721034e-02, -1.50010847e-02, -7.55273784e-03,
        -7.81729817e-03, -6.55076141e-03,  6.98962249e-03,  1.43377725e-02,
        -8.47538933e-03, -5.53713944e-05,  1.22432495e-02, -2.02622749e-02,
        -6.41834661e-02,  2.72127353e-02, -2.12693829e-02,  2.18017735e-02,
         8.99725035e-03,  3.17632519e-02,  7.13081157e-04, -4.82876040e-03,
        -2.71963440e-02,  1.06213370e-03,  3.13878879e-02, -8.19792040e-03,
         1.70046743e-03,  5.33323064e-02,  1.96015373e-01, -2.00279281e-02,
        -6.02360303e-03,  6.01946041e-02, -7.49219907e-03,  7.56498473e-03,
         3.52778239e-03, -4.57342993e-03,  1.15390783e-02,  5.28067467e-04,
        -5.98049089e-02,  1.24754943e-02, -3.81889008e-02,  1.87402591e-02,
        -4.49404074e-03,  8.57056584e-03, -2.94017745e-03,  1.09376367e-02,
         2.43355911e-02,  1.93661712e-02, -1.72345992e-02,  2.49751341e-02,
         2.30827481e-02, -2.40264722e-04,  1.41236801e-02,  1.12972260e-02,
        -8.2

# SVM

In [ ]:
# from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV


# defining parameter range
param_grid = {'C': [0.1, 1, 10, 100, 1000],
              'gamma': [1, 0.1, 0.01, 0.001, 0.0001, 'scale'],
              'kernel': ['rbf', 'linear']}

# grid = GridSearchCV(svm.SVC(), param_grid, refit = True, verbose = 3)
grid = RandomizedSearchCV(svm.SVC(class_weight='balanced'),  param_grid, cv=3, random_state=42, refit = True, verbose = 2)

# fitting the model for grid search
grid.fit(X_train_embeddings, Y_train)
# print best parameter after tuning
print(grid.best_params_)
# print how our model looks after hyper-parameter tuning
print(grid.best_estimator_)

grid_predictions = grid.predict(X_test_embeddings)

# print classification report
print(classification_report(Y_test, grid_predictions))

Fitting 3 folds for each of 10 candidates, totalling 30 fits
[CV] END .........................C=0.1, gamma=1, kernel=rbf; total time=   0.1s
[CV] END .........................C=0.1, gamma=1, kernel=rbf; total time=   0.0s
[CV] END .........................C=0.1, gamma=1, kernel=rbf; total time=   0.0s
[CV] END ...................C=0.1, gamma=0.01, kernel=linear; total time=   0.0s
[CV] END ...................C=0.1, gamma=0.01, kernel=linear; total time=   0.0s
[CV] END ...................C=0.1, gamma=0.01, kernel=linear; total time=   0.0s
[CV] END .........................C=100, gamma=1, kernel=rbf; total time=   0.0s
[CV] END .........................C=100, gamma=1, kernel=rbf; total time=   0.0s
[CV] END .........................C=100, gamma=1, kernel=rbf; total time=   0.0s
[CV] END .................C=100, gamma=0.0001, kernel=linear; total time=   0.0s
[CV] END .................C=100, gamma=0.0001, kernel=linear; total time=   0.0s
[CV] END .................C=100, gamma=0.0001, k

#Random Forest

In [ ]:
# defining parameter range

# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 300, num = 10)]
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]

# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'criterion': ['gini', 'entropy'],
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}


from sklearn.ensemble import RandomForestClassifier
RandomForest = RandomForestClassifier(random_state =0, class_weight='balanced')

grid = RandomizedSearchCV(RandomForest, random_grid, cv=3, random_state=42, refit = True, verbose = 2)
# fitting the model for grid search
grid.fit(X_train_embeddings, Y_train)
# print best parameter after tuning
print(grid.best_params_)
# print how our model looks after hyper-parameter tuning
print(grid.best_estimator_)

grid_predictions = grid.predict(X_test_embeddings)

# print classification report
print(classification_report(Y_test, grid_predictions))

Fitting 3 folds for each of 10 candidates, totalling 30 fits
[CV] END bootstrap=True, criterion=gini, max_depth=100, min_samples_leaf=2, min_samples_split=10, n_estimators=200; total time=   0.7s
[CV] END bootstrap=True, criterion=gini, max_depth=100, min_samples_leaf=2, min_samples_split=10, n_estimators=200; total time=   0.7s
[CV] END bootstrap=True, criterion=gini, max_depth=100, min_samples_leaf=2, min_samples_split=10, n_estimators=200; total time=   0.7s
[CV] END bootstrap=False, criterion=entropy, max_depth=60, min_samples_leaf=4, min_samples_split=10, n_estimators=222; total time=   1.2s
[CV] END bootstrap=False, criterion=entropy, max_depth=60, min_samples_leaf=4, min_samples_split=10, n_estimators=222; total time=   1.2s
[CV] END bootstrap=False, criterion=entropy, max_depth=60, min_samples_leaf=4, min_samples_split=10, n_estimators=222; total time=   1.2s
[CV] END bootstrap=False, criterion=gini, max_depth=110, min_samples_leaf=2, min_samples_split=2, n_estimators=222; tota

## SGDClassifier

In [ ]:
from sklearn.linear_model import SGDClassifier

SGDC = SGDClassifier(class_weight='balanced')

# Create the random grid
random_grid = { "loss" : ["hinge", "log", "squared_hinge", "modified_huber", "perceptron"],
                "alpha" : [0.0001, 0.001, 0.01, 0.1],
                "penalty" : ["l2", "l1", "elasticnet", "none"]}

grid = RandomizedSearchCV(SGDC, random_grid, cv=3, random_state=42, refit = True, verbose = 2)


# fitting the model for grid search
grid.fit(X_train_embeddings, Y_train)
# print best parameter after tuning
print(grid.best_params_)
# print how our model looks after hyper-parameter tuning
print(grid.best_estimator_)

grid_predictions = grid.predict(X_test_embeddings)

# print classification report
print(classification_report(Y_test, grid_predictions))

Fitting 3 folds for each of 10 candidates, totalling 30 fits


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_stochastic_gradient.py:702: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(


[CV] END alpha=0.001, loss=squared_hinge, penalty=elasticnet; total time=   0.5s


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_stochastic_gradient.py:702: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(


[CV] END alpha=0.001, loss=squared_hinge, penalty=elasticnet; total time=   0.6s


...

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_stochastic_gradient.py:702: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(


[CV] END ........alpha=0.001, loss=squared_hinge, penalty=l2; total time=   0.3s
[CV] END alpha=0.0001, loss=squared_hinge, penalty=elasticnet; total time=   0.0s
[CV] END alpha=0.0001, loss=squared_hinge, penalty=elasticnet; total time=   0.1s
[CV] END alpha=0.0001, loss=squared_hinge, penalty=elasticnet; total time=   0.1s
[CV] END ..alpha=0.1, loss=squared_hinge, penalty=elasticnet; total time=   0.1s
[CV] END ..alpha=0.1, loss=squared_hinge, penalty=elasticnet; total time=   0.1s
[CV] END ..alpha=0.1, loss=squared_hinge, penalty=elasticnet; total time=   0.1s
[CV] END .................alpha=0.0001, loss=log, penalty=l2; total time=   0.0s


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_stochastic_gradient.py:163: FutureWarning: The loss 'log' was deprecated in v1.1 and will be removed in version 1.3. Use `loss='log_loss'` which is equivalent.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_stochastic_gradient.py:163: FutureWarning: The loss 'log' was deprecated in v1.1 and will be removed in version 1.3. Use `loss='log_loss'` which is equivalent.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_stochastic_gradient.py:163: FutureWarning: The loss 'log' was deprecated in v1.1 and will be removed in version 1.3. Use `loss='log_loss'` which is equivalent.
  warnings.warn(


[CV] END .................alpha=0.0001, loss=log, penalty=l2; total time=   0.0s
[CV] END .................alpha=0.0001, loss=log, penalty=l2; total time=   0.0s
[CV] END ......alpha=0.0001, loss=modified_huber, penalty=l2; total time=   0.0s
[CV] END ......alpha=0.0001, loss=modified_huber, penalty=l2; total time=   0.0s
[CV] END ......alpha=0.0001, loss=modified_huber, penalty=l2; total time=   0.0s
{'penalty': 'elasticnet', 'loss': 'squared_hinge', 'alpha': 0.0001}
SGDClassifier(class_weight='balanced', loss='squared_hinge',
              penalty='elasticnet')
              precision    recall  f1-score   support

         0.0       0.86      0.86      0.86        57
         1.0       0.85      0.85      0.85        54

    accuracy                           0.86       111
   macro avg       0.86      0.86      0.86       111
weighted avg       0.86      0.86      0.86       111



/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
3 fits failed out of a total of 30.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
3 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_stochastic_gradient.py", line 891, in fit
    self._validate_params()
  File "/usr/local/lib/python3.10/dist-packages/sklearn/base.py", line 600, in _validate_params
    validate_parameter_constraints(
  File "/usr/local/lib/python3.10/dist-package

## XGBoost

In [ ]:
from xgboost import XGBClassifier
from sklearn.metrics import f1_score
from sklearn.utils.class_weight import compute_sample_weight

sample_weights = compute_sample_weight(
    class_weight='balanced',
    y=Y_train
)

xgb = XGBClassifier()

# Create the random grid
random_grid = { "learning_rate" : [0.05,0.1,0.15,0.20],
                 "n_estimators" : [100, 500, 900, 1100, 1500],
                 "max_depth" : [ 3, 5, 6, 10, 15],
                 "min_child_weight" : [ 1, 3, 5, 7 ],
                 "gamma" : [ 0.0, 0.1, 0.2 , 0.3, 0.4 ]}

grid = RandomizedSearchCV(xgb, random_grid, cv=3, random_state=42, refit = True, verbose = 2)

# fitting the model for grid search
grid.fit(X_train_embeddings, Y_train, sample_weight=sample_weights)
# print best parameter after tuning
print(grid.best_params_)
# print how our model looks after hyper-parameter tuning
print(grid.best_estimator_)

grid_predictions = grid.predict(X_test_embeddings)

# print classification report
print(classification_report(Y_test, grid_predictions))

Fitting 3 folds for each of 10 candidates, totalling 30 fits
[CV] END gamma=0.2, learning_rate=0.2, max_depth=5, min_child_weight=3, n_estimators=500; total time=   1.8s
[CV] END gamma=0.2, learning_rate=0.2, max_depth=5, min_child_weight=3, n_estimators=500; total time=   1.2s
[CV] END gamma=0.2, learning_rate=0.2, max_depth=5, min_child_weight=3, n_estimators=500; total time=   1.1s
[CV] END gamma=0.3, learning_rate=0.15, max_depth=6, min_child_weight=7, n_estimators=1500; total time=   1.6s
[CV] END gamma=0.3, learning_rate=0.15, max_depth=6, min_child_weight=7, n_estimators=1500; total time=   1.6s
[CV] END gamma=0.3, learning_rate=0.15, max_depth=6, min_child_weight=7, n_estimators=1500; total time=   1.7s
[CV] END gamma=0.2, learning_rate=0.05, max_depth=10, min_child_weight=1, n_estimators=100; total time=   3.0s
[CV] END gamma=0.2, learning_rate=0.05, max_depth=10, min_child_weight=1, n_estimators=100; total time=   4.5s
[CV] END gamma=0.2, learning_rate=0.05, max_depth=10, min